In [1]:
!pip install jsonlines
!pip install transformers
!pip install simpletransformers


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import jsonlines
import pandas as pd

In [2]:
# Reading train.jsonl file and creating dataframe
dic_train =[]

with jsonlines.open('train.jsonl') as f:
    for line in f.iter():
        dic_train.append(line)  

columns  = dic_train[0].keys()        
df_train = pd.DataFrame(columns=columns)

for i in dic_train:
    df_train.loc[len(df_train.index)] = list(i.values())

In [3]:
df_train.head(1)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,0af11f6b-c889-4520-9372-66ba25cb7657,532quh,"[Wes Welker Wanted Dinner With Tom Brady, But ...",reddit,[It’ll be just like old times this weekend for...,"Wes Welker Wanted Dinner With Tom Brady, But P...",It'll be just like old times this weekend for ...,"new england patriots, ricky doyle, top stories,","[http://pixel.wp.com/b.gif?v=noscript, http://...",http://nesn.com/2016/09/wes-welker-wanted-dinn...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[how about that morning we go throw?],"[[[3, 151], [3, 186]]]",[passage]


In [6]:
#Creating the format to train the model 

train = []
for i in range(df_train.shape[0]):
    entry = df_train.loc[i]
    form = {}
    form['context'] = ' '.join(entry['targetParagraphs'])
    al = []
    uuid = entry['uuid']
    for j in range(len(entry['spoilerPositions'])):
        ids = uuid +' '+ str(i)
        is_impossible = False
        question = entry['targetParagraphs'][entry['spoilerPositions'][j][0][0]]
        answers = [{'text':entry['spoiler'][j], 'answer_start':form['context'].find(entry['spoiler'][j])}]
        al.append({ 'id':ids,'is_impossible':is_impossible,'question':question,'answers':answers})


    form['qas'] = al
    train.append(form)

In [7]:
# Reading validation.jsonl file and creating dataframe
dic_valid =[]

with jsonlines.open('validation.jsonl') as f:
    for line in f.iter():
        dic_valid.append(line)
columns  = dic_valid[0].keys()
df_valid = pd.DataFrame(columns=columns)

for i in dic_valid:
    df_valid.loc[len(df_valid.index)] = list(i.values())

In [8]:
df_valid.head(1)

,uuid,postId,postText,postPlatform,targetParagraphs,targetTitle,targetDescription,targetKeywords,targetMedia,targetUrl,provenance,spoiler,spoilerPositions,tags
0,6dc7ddef-4e8e-4a6b-9296-526377518071,800048986762423_885081814925806,[Five Nights at Freddy’s Sequel Delayed for We...,Facebook,[Five Nights at Freddy’s creator Scott Cawthon...,Five Nights at Freddy’s Sequel Delayed for Wei...,Five Nights at Freddy's creator Scott Cawthon ...,None,[https://gamerant.com/wp-content/uploads/five-...,https://gamerant.com/five-nights-at-freddys-se...,"{'source': 'anonymized', 'humanSpoiler': 'They...",[some of the plot elements are so disturbing t...,"[[[2, 158], [2, 236]]]",[passage]


In [9]:
valid = []
for i in range(df_valid.shape[0]):
    entry = df_valid.loc[i]
    form = {}
    form['context'] = ' '.join(entry['targetParagraphs'])
    al = []
    uuid = entry['uuid']
    for j in range(len(entry['spoilerPositions'])):
        ids = uuid +' '+ str(i)
        is_impossible = False
        question = entry['targetParagraphs'][entry['spoilerPositions'][j][0][0]]
        answers = [{'text':entry['spoiler'][j], 'answer_start':form['context'].find(entry['spoiler'][j])}]
        al.append({ 'id':ids,'is_impossible':is_impossible,'question':question,'answers':answers})

    form['qas'] = al
    valid.append(form)

In [8]:
import torch
from transformers import pipeline

In [9]:
question_answering = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\VETSA TARUN\anaconda3\lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\VETSA TARUN\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [14]:
import logging
from transformers import DistilBertTokenizer, DistilBertForQuestionAnswering
from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForQuestionAnswering.from_pretrained('distilbert-base-uncased')

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

args = QuestionAnsweringArgs()
args.train_batch_size = 16
args.evaluate_during_training = True
args.evaluate_during_training_steps = 1000
args.save_steps = -1

# Initialize the QuestionAnsweringModel
model = QuestionAnsweringModel(
    'distilbert', 
    'distilbert-base-uncased', 
    args=args,
    use_cuda=False
)


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this mode

In [16]:
import shutil
import os

# specify the directory to delete
output_dir = 'outputs/'

# delete the directory and its contents
shutil.rmtree(output_dir)

# create a new output directory
os.mkdir(output_dir)


In [17]:
model.train_model(train, eval_data=valid)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|█████████████████████████████████████████| 4622/4622 [00:00<00:00, 278167.84it/s]


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/729 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.

convert squad examples to features: 100%|██████████████████████████████████████████| 1192/1192 [00:27<00:00, 43.92it/s]

add example index and unique id: 100%|██████████████████████████████████████████| 1192/1192 [00:00<00:00, 62854.18it/s]


Running Evaluation:   0%|          | 0/408 [00:00<?, ?it/s]

INFO:simpletransformers.question_answering.question_answering_model: Training of distilbert model complete. Saved to outputs/.


(729,
 {'global_step': [729],
  'correct': [35],
  'similar': [620],
  'incorrect': [145],
  'train_loss': [0.0022680554538965225],
  'eval_loss': [-5.345833968298108]})

In [18]:
# Evaluate the model
fin_res, _ = model.eval_model(valid)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|█████████████████████████████████████████████████████| 1192/1192 [00:00<?, ?it/s]


Running Evaluation:   0%|          | 0/408 [00:00<?, ?it/s]

In [21]:
print('Final Result:',fin_res )

Final Result: {'correct': 35, 'similar': 620, 'incorrect': 145, 'eval_loss': -5.345833968298108}


In [19]:
print('ACCURACY:',((fin_res[0]+finres[1])/(fin_res[0]+fin_res[1]+fin_res[2]))*100,'%')

ACCURACY: 81.875 %


In [20]:
all_ans, prob = model.predict(valid)

INFO:simpletransformers.question_answering.question_answering_model: Converting to features started.
add example index and unique id: 100%|█████████████████████████████████████████| 1192/1192 [00:00<00:00, 138309.46it/s]


Running Prediction:   0%|          | 0/408 [00:00<?, ?it/s]

In [21]:
# Extracting final prediction based on all probabilities of all answers

predicted =[]
a = all_ans[0]['id'].split()[0]

if all_ans[0]['answer'][0] =='':
    pred = [all_ans[0]['answer'][1]]
else:
    pred = [all_ans[0]['answer'][0]]  

for id in all_ans[1:]:  
    if a == id['id'].split()[0]:
        if id['answer'][0] =='':
            pred.extend([id['answer'][1]])
        else:  
            pred.extend([id['answer'][0]])  
    else:
        predicted.append(pred)
        if id['answer'][0]=='':
            pred = [id['answer'][1]]
        else:
            pred = [id['answer'][0]]

    a = id['id'].split()[0] 
predicted.append(pred)  


In [22]:
fin_out={'uuid':list(df_valid['uuid']),'Predicted_spoiler':predicted}

In [23]:
fin_df=pd.DataFrame(fin_out)

In [24]:
fin_df

,uuid,Predicted_spoiler
0,6dc7ddef-4e8e-4a6b-9296-526377518071,[Five Nights]
1,435b24de-56f6-4d4e-9c38-54b8e0630aac,[© REUTERS/Laura Segall Maricopa County Sherif...
2,8091ba93-6376-473a-9117-01d6cf0507e4,[hairdresserMore Remembering how much you’re s...
3,8b713277-0e5b-4873-a216-b650f21f3b4c,[CBGB. Opened in 1973 in downtown Manhattan's ...
4,a2f91b65-c36c-481f-92b1-2fc77d6411fc,"[Also, they just ran a story about a man who s..."
...,...,...
795,1189d343-42eb-47e7-8395-ff978a683875,[hotel cleaner]
796,7912282b-137b-4098-875d-8ad9f19354a8,"[Christopher Suprun,]"
797,1fdf71e8-ec14-4c3b-a7c5-ca678c6f8ccb,[What]
798,17f6b540-cf8d-4ddf-8321-1c9ce2315d71,[Julian Assange’s internet link has been inten...
